In [2]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster

from utils import *
import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib

from scipy.ndimage import label

import json
import pickle

# rsa toolbox imports
from rsatoolbox.io.mne import read_epochs
from rsatoolbox.data.ops import merge_datasets
from rsatoolbox.rdm import calc_rdm_movie
from rsatoolbox.rdm.calc import _parse_input
from rsatoolbox.util.build_rdm import _build_rdms
from rsatoolbox.rdm import compare
from rsatoolbox.vis import show_rdm
from rsatoolbox.vis.timecourse import plot_timecourse

from os.path import join, expanduser, basename
import glob, json
import numpy, tqdm, mne, pandas
import rsatoolbox
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot

['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin', 'C:/Users/j

In [3]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
filename = 'subjects_electrodestoROIs_dict.json'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_rois_dict(filename, subjects)

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


In [4]:
stimulus_locked = True  #toggle
response_locked = not stimulus_locked

if stimulus_locked:
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8"
elif response_locked:
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8"

In [5]:
stimulus_experiment_conditions = {
    "Stimulus/i25.0/s25.0": {
        "BIDS_events": "Stimulus/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/i25.0/s75.0": {
        "BIDS_events": "Stimulus/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/i75.0/s25.0": {
        "BIDS_events": "Stimulus/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/i75.0/s75.0": {
        "BIDS_events": "Stimulus/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/i25.0/r25.0": {
        "BIDS_events": "Stimulus/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/i25.0/r75.0": {
        "BIDS_events": "Stimulus/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/i75.0/r25.0": {
        "BIDS_events": "Stimulus/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/i75.0/r75.0": {
        "BIDS_events": "Stimulus/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/c25.0/s25.0": {
        "BIDS_events": "Stimulus/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/c25.0/s75.0": {
        "BIDS_events": "Stimulus/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/c75.0/s25.0": {
        "BIDS_events": "Stimulus/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/c75.0/s75.0": {
        "BIDS_events": "Stimulus/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/c25.0/r25.0": {
        "BIDS_events": "Stimulus/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/c25.0/r75.0": {
        "BIDS_events": "Stimulus/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/c75.0/r25.0": {
        "BIDS_events": "Stimulus/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/c75.0/r75.0": {
        "BIDS_events": "Stimulus/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

# congruency_conditions = {
#     "Stimulus_c": {
#         "BIDS_events": ["Stimulus/c25/s25", "Stimulus/c25/s75", "Stimulus/c75/s25", "Stimulus/c75/s75", "Stimulus/c25/r25", "Stimulus/c25/r75", "Stimulus/c75/r25", "Stimulus/c75/r75"],
#         "congruency": "c"
#     }
# }

stimulus_conditions = {
    "Stimulus/BigLetters/SmallLetterh/Taskg": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus/BigLetters/SmallLetterh/Taskl": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus/BigLetters/SmallLetters/Taskg": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus/BigLetters/SmallLetters/Taskl": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Stimulus/BigLetterh/SmallLetterh/Taskg": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus/BigLetterh/SmallLetterh/Taskl": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus/BigLetterh/SmallLetters/Taskg": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus/BigLetterh/SmallLetters/Taskl": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

response_experiment_conditions = {
    "Response/i25.0/s25.0": {
        "BIDS_events": "Response/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/i25.0/s75.0": {
        "BIDS_events": "Response/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/i75.0/s25.0": {
        "BIDS_events": "Response/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/i75.0/s75.0": {
        "BIDS_events": "Response/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/i25.0/r25.0": {
        "BIDS_events": "Response/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/i25.0/r75.0": {
        "BIDS_events": "Response/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/i75.0/r25.0": {
        "BIDS_events": "Response/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/i75.0/r75.0": {
        "BIDS_events": "Response/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/c25.0/s25.0": {
        "BIDS_events": "Response/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/c25.0/s75.0": {
        "BIDS_events": "Response/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/c75.0/s25.0": {
        "BIDS_events": "Response/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/c75.0/s75.0": {
        "BIDS_events": "Response/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/c25.0/r25.0": {
        "BIDS_events": "Response/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/c25.0/r75.0": {
        "BIDS_events": "Response/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/c75.0/r25.0": {
        "BIDS_events": "Response/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/c75.0/r75.0": {
        "BIDS_events": "Response/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

response_conditions = {
    "Response/BigLetters/SmallLetterh/Taskg": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Response/BigLetters/SmallLetterh/Taskl": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Response/BigLetters/SmallLetters/Taskg": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Response/BigLetters/SmallLetters/Taskl": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Response/BigLetterh/SmallLetterh/Taskg": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Response/BigLetterh/SmallLetterh/Taskl": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Response/BigLetterh/SmallLetters/Taskg": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Response/BigLetterh/SmallLetters/Taskl": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

In [6]:
task='GlobalLocal'

if stimulus_locked:
    conditions = stimulus_conditions
elif response_locked:
    conditions = response_conditions #set this to whichever you're running

condition_names = list(conditions.keys()) # get the condition names as a list

# Assuming 'combined_data' is your DataFrame and 'subjects' is your list of subject IDs
subjects_mne_objects = create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=True)

Loading data for subject: D0057
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated
  Loading condition: Stimulus/BigLetters/SmallLetterh/Taskg with parameters: {'BIDS_events': 'Stimulus/BigLetters/SmallLetterh/Taskg', 'bigLetter': 's', 'smallLetter': 'h', 'task': 'g'}
  Loading condition: Stimulus/BigLetters/SmallLetterh/Taskl with parameters: {'BIDS_events': 'Stimulus/BigLetters/SmallLetterh/Taskl', 'bigLetter': 's', 'smallLetter': 'h', 'task': 'l'}
  Loading condition: Stimulus/BigLetters/SmallLetters/Taskg with parameters: {'BIDS_events': 'Stimulus/BigLetters/SmallLetters/Taskg', 'bigLetter': 's', 'smallLetter': 's', 'ta

In [7]:
condition_names = list(conditions.keys()) # get the condition names as a list
condition_names_bids = [condition['BIDS_events'] for condition in conditions.values()] # get the condition names in bids format
condition_names

['Stimulus/BigLetters/SmallLetterh/Taskg',
 'Stimulus/BigLetters/SmallLetterh/Taskl',
 'Stimulus/BigLetters/SmallLetters/Taskg',
 'Stimulus/BigLetters/SmallLetters/Taskl',
 'Stimulus/BigLetterh/SmallLetterh/Taskg',
 'Stimulus/BigLetterh/SmallLetterh/Taskl',
 'Stimulus/BigLetterh/SmallLetters/Taskg',
 'Stimulus/BigLetterh/SmallLetters/Taskl']

get significant channels

In [8]:
sig_chans_per_subject = get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


In [9]:
# Assuming you have a list of subjects
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
root_dir = rf"C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs"


In [10]:
# Define your ROIs dictionary and other parameters
rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
}

rois = list(rois_dict.keys())
# Assuming you have subjects_electrodestoROIs_dict and sig_chans_per_subject dictionaries
electrodes_per_subject_roi, sig_electrodes_per_subject_roi,  = make_sig_electrodes_per_subject_and_roi_dict(
    rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject
)


For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI15', 'RAI16', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LOF16', 'LASF10', 'LASF14', 'LASF15', 'LASF16', 'LM

get electrode counts for each roi

In [11]:
total_electrodes_info = calculate_total_electrodes(sig_electrodes_per_subject_roi, electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])


Total number of lpfc electrodes across all subjects: 105
Total number of significant lpfc electrodes across all subjects: 46
Total number of v1 electrodes across all subjects: 17
Total number of significant v1 electrodes across all subjects: 12
Total number of occ electrodes across all subjects: 57
Total number of significant occ electrodes across all subjects: 41


check if any subjects have a weird sampling rate

In [12]:
# Assuming 'subjects_mne_objects' is your dictionary containing MNE objects for each subject
sampling_rate = 256
subject_rates = check_sampling_rates(subjects_mne_objects, expected_sampling_rate=sampling_rate)


Subject D0057 has the expected sampling rate: 256.0 Hz.
Subject D0059 has the expected sampling rate: 256.0 Hz.
Subject D0063 has the expected sampling rate: 256.0 Hz.
Subject D0065 has the expected sampling rate: 256.0 Hz.
Subject D0069 has the expected sampling rate: 256.0 Hz.
Subject D0071 has the expected sampling rate: 256.0 Hz.
Subject D0077 has the expected sampling rate: 256.0 Hz.
Subject D0090 has the expected sampling rate: 256.0 Hz.
Subject D0094 has the expected sampling rate: 256.0 Hz.
Subject D0100 has the expected sampling rate: 256.0 Hz.
Subject D0102 has the expected sampling rate: 256.0 Hz.
Subject D0103 has the expected sampling rate: 256.0 Hz.


create a dat dict like in the Temporal example from rsatoolbox 7/19  
#### Data Dictionary (dat) Structure
- **roi1**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roi2**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roiX**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample


        




In [13]:
sig_electrodes = sig_electrodes_per_subject_roi['lpfc'].get('D0059', [])
sig_electrodes

['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']

In [15]:
import numpy as np

def prepare_data_for_temporal_dataset(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi):
    # returns dat
    dat = {}
    overall_electrode_mapping = []
    electrode_mapping_per_roi = {roi: [] for roi in rois}  # Reinitialize for each processing run
    print('subjects: ', subjects)
    for roi in rois:
        dat[roi] = {}  # make a dict for each roi
        dat[roi]['channel_names'] = []  # initialize a list to hold channel names
        dat[roi]['channel_rois'] = [] # initialize a list to hold what roi each channel is a part of
        dat[roi]['condition_names'] = {}  # initialize dict where keys are condition names and values are integer indices
        dat[roi]['cond_idx'] = np.array([], dtype=int)  # initialize an empty 1D array for condition indices for each trial
        dat[roi]['sub_idx'] = np.array([], dtype=int) # initialize an empty 1D array for subject for each trial
        dat[roi]['times'] = np.array([])  # initialize an empty 1D array for time points

        # Determine all unique channels across subjects for this ROI, maintaining order
        all_channels = []
        for sub in subjects:
            sig_electrodes = sig_electrodes_per_subject_roi[roi].get(sub, [])
            sub_channel_names = [sub + '-' + sig_electrode for sig_electrode in sig_electrodes]
            for chan in sub_channel_names:
                if chan not in all_channels:
                    all_channels.append(chan)
                    dat[roi]['channel_rois'].append(roi) # append roi for each channel
        dat[roi]['channel_names'] = all_channels
        num_channels = len(all_channels)
        print('num channels: ', num_channels)

        # Initialize the data array with the number of trials and total channels
        dat[roi]['data'] = np.empty((0, num_channels, 0))  # initialize an empty 3D array for trials x channels x time points

        for sub in subjects:
            total_trials = 0  # Initialize counter for total trials across all conditions
            sig_electrodes = sig_electrodes_per_subject_roi[roi].get(sub, [])
            sub_channel_names = [sub + '-' + sig_electrode for sig_electrode in sig_electrodes]
            if not sig_electrodes:
                continue

            cond_idx = 0  # the example uses indexing from 1, but let's start from 0 because python
            for condition_name in condition_names:
                print(f'Processing {sub} for {condition_name} in {roi}')
                epochs = subjects_mne_objects[sub][condition_name]['HG_ev1_rescaled'].copy().pick(sig_electrodes)
                dat[roi]['condition_names'][condition_name] = cond_idx

                epochs_data = epochs.get_data(copy=True)
                num_trials, num_sub_channels, num_timepoints = epochs_data.shape

                print(f'Number of trials for {sub} in {condition_name}: {num_trials}')
                total_trials += num_trials

                # Initialize data array time dimension if it is empty
                if dat[roi]['data'].shape[2] == 0:
                    dat[roi]['data'] = np.empty((0, num_channels, num_timepoints))
                    dat[roi]['times'] = epochs.times

                # Create an array filled with NaNs for the current subject's data
                sub_data = np.full((num_trials, num_channels, num_timepoints), np.nan)

                # Find the indices for the subject's channels in the total list of channels
                channel_indices = [all_channels.index(chan) for chan in sub_channel_names]
                print('sub: ', sub)
                print("channel indices: ", channel_indices)
                
                # Place the subject's data in the correct indices
                sub_data[:, channel_indices, :] = epochs_data

                # Concatenate the new data along the first axis (trials)
                dat[roi]['data'] = np.concatenate((dat[roi]['data'], sub_data), axis=0)

                # Extend the cond_idx array
                dat[roi]['cond_idx'] = np.concatenate((dat[roi]['cond_idx'], np.full(num_trials, cond_idx)))

                # extend the sub_idx array by this number of trials with this subject
                dat[roi]['sub_idx'] = np.concatenate((dat[roi]['sub_idx'], np.full(num_trials, sub)))
                print('last entry in cond_idx is: ', dat[roi]['cond_idx'][-1])
                print('cond_idx is: ', cond_idx)
                cond_idx += 1  # increment cond_idx

            print(f'Total number of trials for {sub} across all conditions: {total_trials}')

    return dat

# Example call to the function (you need to replace the arguments with actual data)
dat = prepare_data_for_temporal_dataset(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi)


subjects:  ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
num channels:  46
Processing D0057 for Stimulus/BigLetters/SmallLetterh/Taskg in lpfc
Number of trials for D0057 in Stimulus/BigLetters/SmallLetterh/Taskg: 59
sub:  D0057
channel indices:  [0, 1]
last entry in cond_idx is:  0
cond_idx is:  0
Processing D0057 for Stimulus/BigLetters/SmallLetterh/Taskl in lpfc
Number of trials for D0057 in Stimulus/BigLetters/SmallLetterh/Taskl: 54
sub:  D0057
channel indices:  [0, 1]
last entry in cond_idx is:  1
cond_idx is:  1
Processing D0057 for Stimulus/BigLetters/SmallLetters/Taskg in lpfc
Number of trials for D0057 in Stimulus/BigLetters/SmallLetters/Taskg: 53
sub:  D0057
channel indices:  [0, 1]
last entry in cond_idx is:  2
cond_idx is:  2
Processing D0057 for Stimulus/BigLetters/SmallLetters/Taskl in lpfc
Number of trials for D0057 in Stimulus/BigLetters/SmallLetters/Taskl: 34
sub:  D0057
channel indices:  [0, 1]
last entry i

plot condition averages for two channels in occ

In [16]:
# Extract the relevant data from the dictionary
occ_chan_names = dat['occ']['channel_names']
occ_data = dat['occ']['data']
occ_cond_idx = dat['occ']['cond_idx']
occ_times = dat['occ']['times']
occ_cond_names = [x for x in dat['occ']['condition_names'].keys()]

# Create the subplots
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
ax = ax.flatten()

# Plot the data
for jj, chan in enumerate(occ_chan_names[:2]):
    for ii, cond_ii in enumerate(np.unique(occ_cond_idx)):
        mn = np.nanmean(occ_data[cond_ii == occ_cond_idx, jj, :], axis=0).squeeze()
        ax[jj].plot(occ_times, mn, label=occ_cond_names[ii])
        ax[jj].set_title(chan)

# Add the legend
ax[0].legend()
# plt.show()


turn the power trace plotting into a function 7/21

In [17]:
# Function to compare conditions and save the plots
def compare_conditions_and_save(dat, rois, save_dir, colors, linestyles, conditions_to_be_plotted):
    for roi in rois:
        # Extract the relevant data for the current ROI
        roi_chan_names = dat[roi]['channel_names']
        roi_data = dat[roi]['data']
        roi_cond_idx = dat[roi]['cond_idx']
        roi_times = dat[roi]['times']
        roi_cond_names = [x for x in dat[roi]['condition_names'].keys()]

        for label, condition_dict in conditions_to_be_plotted.items():
            plt.figure(figsize=(12,6))
            for cond_name, cond_values in condition_dict.items():
                if isinstance(cond_values, str):
                    cond_values = [cond_values]
                    
                # Group condition indices by whether they contain any of the specified strings in the list
                conditions = [name for name in roi_cond_names if any(c in name for c in cond_values)]
                condition_indices = [dat[roi]['condition_names'][name] for name in conditions]

                # Calculate the mean and standard error over conditions for each group
                condition_data = np.concatenate([roi_data[roi_cond_idx == idx, :, :] for idx in condition_indices], axis=0)

                # Calculate the mean over trials for each group and channel
                mean_condition = np.nanmean(condition_data, axis=0)

                # Calculate standard error over the trials for each channel and time point
                sem_condition = np.nanstd(condition_data, axis=0, ddof=1) / np.sqrt(condition_data.shape[0])

                # Average the mean and SEM across channels for plotting
                mean_condition = np.nanmean(mean_condition, axis=0)
                sem_condition = np.nanmean(sem_condition, axis=0)

                # Plot the results with standard error shading
                plt.plot(roi_times, mean_condition, label=cond_name, color=colors[label], linestyle=linestyles[cond_name])
                plt.fill_between(roi_times, mean_condition - sem_condition, mean_condition + sem_condition, color=colors[label], alpha=0.3)

            plt.xlabel('Time from Stim Onset (s)')
            plt.ylabel('Z-score')
            plt.title(f'{roi} Average over conditions')
            plt.legend()
            plt.ylim([-1, 1])

            # Save the figure
            filename = f'{roi}_{label}_rsatoolbox_power_plot.png'
            plt.savefig(os.path.join(save_dir, filename))
            plt.close()

In [18]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")
save_dir = os.path.join(LAB_root, 'BIDS-1.1_GlobalLocal', 'BIDS', 'derivatives', 'RSA', 'figs')
print(save_dir)
rois = ['lpfc', 'v1', 'occ']

# Define colors for the model names
colors = {
    'congruency': 'red',
    'switchType': 'blue',
    'congruencyProportion': 'pink',
    'switchProportion': 'skyblue',
    'congruency_congruencyProportion': 'hotpink',
    'switchType_switchProportion': 'gray',
    'bigLetter': 'green',
    'smallLetter': 'orange',
    'task': 'gray',
}

# Define linestyles for the model names
linestyles = {
    'big letter S': '-',
    'big letter H': '--',
    'small letter S': '-.',
    'small letter H': ':',
    'task G': '-',
    'task L': '--',
    'congruent': '-',
    'incongruent': '--',
}

if conditions == stimulus_conditions:
    conditions_to_be_plotted = {
        'bigLetter': {'big letter S': 'BigLetters', 'big letter H': 'BigLetterh'},
        'smallLetter': {'small letter S': 'SmallLetters', 'small letter H': 'SmallLetterh'},
        'task': {'task G': 'Taskg', 'task L': 'Taskl'},
    }
elif conditions == stimulus_experiment_conditions:

    conditions_to_be_plotted = {
        'congruency': {'congruent': ['c25', 'c75'], 'incongruent': ['i25', 'i75']},
        'switchType': {'switch': ['s25', 's75'], 'repeat': ['r25', 'r75']}
    }

compare_conditions_and_save(dat, rois, save_dir, colors, linestyles, conditions_to_be_plotted)

C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\RSA\figs


set up TemporalDataset objects

In [19]:
temporal_datasets = {}

for roi in rois:
    measurements = dat[roi]['data']
    des = {'roi': roi}
    obs_des = {'cond_idx': dat[roi]['cond_idx'], 'sub_idx': dat[roi]['sub_idx']}
    chn_des = {'channel_names': dat[roi]['channel_names'], 'channel_rois': dat[roi]['channel_rois']}
    tim_des = {'time': dat[roi]['times']}

    temporal_datasets[roi] = rsatoolbox.data.TemporalDataset(measurements, 
                                             descriptors = des, 
                                             obs_descriptors = obs_des, 
                                             channel_descriptors = chn_des,
                                             time_descriptors = tim_des)


bin the times (just for occ for now, go back and make this loop over rois) 7/21

In [20]:
import numpy as np

# Assuming temporal_datasets is a dictionary
first_roi = next(iter(temporal_datasets.keys()))
time_points = temporal_datasets[first_roi].time_descriptors['time']

# Define the bins: 100 ms each
bin_size = 0.1  # 100 ms
bins = []

# Find the start and end times from the time_points array
start_time = np.min(time_points)
end_time = np.max(time_points)

current_time = start_time
while current_time < end_time:
    next_time = current_time + bin_size
    if next_time > end_time:
        next_time = end_time
    bins.append(time_points[(time_points >= current_time) & (time_points < next_time)])
    current_time = next_time

# Ensure that the last bin includes the endpoint
if time_points[-1] not in bins[-1]:
    bins[-1] = np.append(bins[-1], time_points[-1])


print("First bin:", bins[0])
print("Last bin:", bins[-1])


First bin: [-1.         -0.99609375 -0.9921875  -0.98828125 -0.984375   -0.98046875
 -0.9765625  -0.97265625 -0.96875    -0.96484375 -0.9609375  -0.95703125
 -0.953125   -0.94921875 -0.9453125  -0.94140625 -0.9375     -0.93359375
 -0.9296875  -0.92578125 -0.921875   -0.91796875 -0.9140625  -0.91015625
 -0.90625    -0.90234375]
Last bin: [1.40234375 1.40625    1.41015625 1.4140625  1.41796875 1.421875
 1.42578125 1.4296875  1.43359375 1.4375     1.44140625 1.4453125
 1.44921875 1.453125   1.45703125 1.4609375  1.46484375 1.46875
 1.47265625 1.4765625  1.48046875 1.484375   1.48828125 1.4921875
 1.49609375 1.5       ]


try make rdm (just do occ for testing)

average the dissimilarities in whatever groups you want (i.e., congruent vs congruent, incongruent vs congruent, incongruent vs incongruent)

In [21]:
# rdms_occ_data = calc_rdm_movie(temporal_datasets['occ'], method='correlation', descriptor='cond_idx', bins=bins, unbalanced=True)

neural_rdms = {}

for roi in rois:
    neural_rdms[roi] = calc_rdm_movie(temporal_datasets[roi], method='correlation', descriptor='cond_idx', bins=bins, unbalanced=True)

    plt.figure(figsize=(10,15))

    # add formatted time as rdm_descriptor
    neural_rdms[roi].rdm_descriptors['time_formatted'] = ['%0.0f ms' % (np.round(x*1000,2)) for x in neural_rdms[roi].rdm_descriptors['time']]


    show_rdm(neural_rdms[roi], 
                    pattern_descriptor='cond_idx',
                    rdm_descriptor='time_formatted', 
                    vmin=-1, vmax=1,
                    show_colorbar='figure')
    
    plt.savefig(os.path.join(save_dir, f'rsatoolbox_visual_rdms_{roi}.png'))
    plt.close()

separate the neural rdms into its different comparisons and plot over time 7/29

In [22]:
colored_conditions = {}

if conditions == stimulus_conditions:
    big_letter = np.array([condition_name.split('/')[1] for condition_name in condition_names])
    small_letter = np.array([condition_name.split('/')[2] for condition_name in condition_names])
    task = np.array([condition_name.split('/')[3] for condition_name in condition_names])

    colored_conditions['big_letter'] = big_letter
    colored_conditions['small_letter'] = small_letter
    colored_conditions['task'] = task

for roi in rois:
    for colored_condition_name, data in colored_conditions.items():
        fig, ax = plot_timecourse(
            neural_rdms[roi], 
            descriptor='cond_idx', 
            n_t_display=10, 
            fig_width=20,
            colored_conditions=data
        )

        # Ensure `ax` is a list or array of axes
        if isinstance(ax, (list, np.ndarray)):
            for a in ax:
                a.set_ylim([-1, 1])
            ax[0].set_title(f'dissimilarity over time in {roi} grouped by {colored_condition_name}')
        else:
            ax.set_ylim([-1, 1])
            ax.set_title(f'dissimilarity over time in {roi} grouped by {colored_condition_name}')

        plt.savefig(os.path.join(save_dir, f'{roi}_{colored_condition_name}_neural_rdm_over_time.png'))
        plt.close()
        # plt.show()

make model rdms

In [23]:
# Define the conditions
def make_model_rdms():
    experiment_conditions_list = ["i25s25", "i25s75", "i75s25", "i75s75", "i25r25", "i25r75", "i75r25", "i75r75",
                "c25s25", "c25s75", "c75s25", "c75s75", "c25r25", "c25r75", "c75r25", "c75r75"]
    stimulus_conditions_list = ["bigSsmallHtaskG", "bigSsmallHtaskL", "bigSsmallStaskG", "bigSsmallStaskL", 
                        "bigHsmallHtaskG", "bigHsmallHtaskL", "bigHsmallStaskG", "bigHsmallStaskL"]

    # Extract specific features from each condition
    congruency = [cond[0] for cond in experiment_conditions_list]
    switchTypes = [cond[-3] for cond in experiment_conditions_list]
    congruencyProportion = [int(cond[1:3]) for cond in experiment_conditions_list]
    switchProportion = [int(cond[-2:]) for cond in experiment_conditions_list]

    bigLetter = [cond[3] for cond in stimulus_conditions_list]
    smallLetter = [cond[9] for cond in stimulus_conditions_list]
    task = [cond[-1] for cond in stimulus_conditions_list]

    # Number of conditions
    e = len(experiment_conditions_list)
    s = len(stimulus_conditions_list)

    experiment_model_rdms_np = {'congruency': np.ones((e,e)), 'switchType': np.ones((e,e)),
                'congruencyProportion': np.ones((e,e)), 'switchProportion': np.ones((e,e)), 
                'congruency_congruencyProportion': np.ones((e,e)), 'switchType_switchProportion': np.ones((e,e))}

    stimulus_model_rdms_np = {'bigLetter': np.ones((s,s)), 'smallLetter': np.ones((s,s)), 'task': np.ones((s,s))}

    # Populate RDMs based on feature comparisons for experimental conditions
    for i in range(e):
        for j in range(e):
            if congruency[i] == congruency[j]:
                experiment_model_rdms_np['congruency'][i,j] = 0
            if switchTypes[i] == switchTypes[j]:
                experiment_model_rdms_np['switchType'][i,j] = 0
            if congruencyProportion[i] == congruencyProportion[j]:
                experiment_model_rdms_np['congruencyProportion'][i,j] = 0
            if switchProportion[i] == switchProportion[j]:
                experiment_model_rdms_np['switchProportion'][i,j] = 0
            if congruency[i] == congruency[j] and congruencyProportion[i] == congruencyProportion[j]:
                experiment_model_rdms_np['congruency_congruencyProportion'][i,j] = 0
            if switchTypes[i] == switchTypes[j] and switchProportion[i] == switchProportion[j]:
                experiment_model_rdms_np['switchType_switchProportion'][i,j] = 0
            # if congruency[i] == congruency[j] and switchProportion[i] == switchProportion[j]:
            #     model_rdms_np['congruency_switchProportion'][i,j] = 0
            # if switchTypes[i] == switchTypes[j] and congruencyProportion[i] == congruencyProportion[j]:
            #     model_rdms_np['switchType_congruencyProportion'][i,j] = 0

    # make model rdms for stimulus details
    for i in range(s):
        for j in range(s):
            if bigLetter[i] == bigLetter[j]:
                stimulus_model_rdms_np['bigLetter'][i,j] = 0
            if smallLetter[i] == smallLetter[j]:
                stimulus_model_rdms_np['smallLetter'][i,j] = 0
            if task[i] == task[j]:
                stimulus_model_rdms_np['task'][i,j] = 0
                
    return experiment_model_rdms_np, stimulus_model_rdms_np

In [24]:
experiment_model_rdms_np, stimulus_model_rdms_np = make_model_rdms()

experiment_model_rdms_list = []
for model_name, model_rdm in experiment_model_rdms_np.items():
    experiment_model_rdms_list.append(rsatoolbox.model.ModelFixed(model_name, model_rdm))

stimulus_model_rdms_list = []
for model_name, model_rdm in stimulus_model_rdms_np.items():
    stimulus_model_rdms_list.append(rsatoolbox.model.ModelFixed(model_name, model_rdm)) 


# stimulus_model_rdms_list[0].name
# pred = stimulus_model_rdms_list[0].predict()
# pred
# # pred = model.predict()
# # pred_rdm = model.predict_rdm()

# # pred_rdm

do model rdm vs neural rdm similarities

In [25]:
def get_rsa_results(model_rdms_list, neural_rdms, rois, method='corr'):
    '''
    Compute RSA (Representational Similarity Analysis) results for each ROI using a specified method.

    Parameters:
    - model_rdms_list (list): List of model RDMs to compare against neural RDMs.
    - neural_rdms (dict): Dictionary where keys are ROIs and values are neural RDMs.
    - rois (list): List of ROIs to analyze.
    - method (str): Method to use for RSA (default is 'corr' for correlation).

    Returns:
    - rsa_results (dict): Dictionary with ROIs as keys and RSA results as values.
    - rsa_results_squeezed (dict): Dictionary with ROIs as keys and RSA results with the 0th dimension squeezed out.
    '''
    rsa_results = {}
    rsa_results_squeezed = {}

    for roi in rois:
        rsa_results[roi] = rsatoolbox.inference.eval_fixed(model_rdms_list, neural_rdms[roi], method=method)
        rsa_results_squeezed[roi] = np.mean(rsa_results[roi].evaluations, axis=0).squeeze()

    return rsa_results, rsa_results_squeezed

def plot_rsa_results(model_rdms_list, neural_rdms, rsa_results_squeezed, rois, save_dir, models_filename):
    '''
    Plot RSA results for each ROI and save the plots with the correct model filename.

    Parameters:
    - model_rdms_list (list): List of model RDMs used in the analysis.
    - neural_rdms (dict): Dictionary where keys are ROIs and values are neural RDMs.
    - rsa_results_squeezed (dict): Dictionary with ROIs as keys and squeezed RSA results as values.
    - rois (list): List of ROIs to plot.
    - save_dir (str): Directory where the plots will be saved.
    - models_filename (str): Filename to use for saving the plots.
    '''
    for roi in rois:
        
        # Create a new plot to avoid any residual plotting issues
        fig, ax = plt.subplots(1, 1, figsize=(20, 6))
        times = neural_rdms[roi].rdm_descriptors['time'] # this assumes all rois have the same times. Will break if not.

        # Plot each model's mean similarity over time
        for i, ev in enumerate(rsa_results_squeezed[roi]):
            model_name = model_rdms_list[i].name
            color = colors.get(model_name, 'black')  # Use black as a default color if not specified
            ax.plot(times, ev, color=color, label=f"model: {model_name}", linewidth=3)

        # Set y-axis limits
        ax.set_ylim(-1, 1)

        # Set plot labels and legend
        ax.legend()
        ax.set_ylabel(f'{roi} Neural RDM to Model RDM Similarity (Corr)')
        ax.set_xlabel('Time')
        ax.set_title(f'{roi} Model-Neural Similarity Over Time')

        # Construct the filename for saving
        filename = f'{roi}_{models_filename}.png'
        plt.savefig(os.path.join(save_dir, filename))
        plt.close()

def get_and_plot_rsa_results(model_rdms_list, neural_rdms, rois, save_dir, models_filename, method='corr'):
    '''
    Get and plot the RSA results for each ROI and save the plots with the correct model filename.

    Parameters:
    - model_rdms_list (list): List of model RDMs used in the analysis.
    - neural_rdms (dict): Dictionary where keys are ROIs and values are neural RDMs.
    - rois (list): List of ROIs to plot.
    - save_dir (str): Directory where the plots will be saved.
    - models_filename (str): Filename to use for saving the plots.
    - method (str): Method to use for RSA (default is 'corr' for correlation).

    Returns:
    - rsa_results (dict): Dictionary with ROIs as keys and RSA results as values.
    - rsa_results_squeezed (dict): Dictionary with ROIs as keys and RSA results with the 0th dimension squeezed out.
    '''
    rsa_results, rsa_results_squeezed = get_rsa_results(model_rdms_list, neural_rdms, rois, method=method)
    plot_rsa_results(model_rdms_list, neural_rdms, rsa_results_squeezed, rois, save_dir, models_filename)
    return rsa_results, rsa_results_squeezed

In [30]:
if conditions == stimulus_conditions:
    models_filename = 'stimulus_models_rsatoolbox_rsa_results'
    rsa_results, rsa_results_squeezed = get_and_plot_rsa_results(stimulus_model_rdms_list, neural_rdms, rois, save_dir, models_filename)

elif conditions == 'experiment_conditions':
    models_filename = 'experiment_models_rsatoolbox_rsa_results'
    rsa_results, rsa_results_squeezed = get_and_plot_rsa_results(experiment_model_rdms_list, neural_rdms, rois, save_dir, models_filename)

In [27]:
# rsatoolbox.vis.plot_model_comparison(stimulus_rsa_results['occ'])
# plt.show()

In [29]:
def debugger_test():
    print('line 1')
    print('line 2')

debugger_test()

line 1
line 2


KeyboardInterrupt: 